In [1]:
import sqlite3 # help us to read sql data 
import pandas as pd 
import numpy as np
import  nltk
import string
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import confusion_matrix
from sklearn import metrics
from sklearn.metrics import roc_curve, auc
from nltk.stem.porter import PorterStemmer

# nltk--> a very popular NLP toolkit to process text.

# using the sql table to read data .
con = sqlite3.connect('C:/Data sets/amazon-fine-food-reviews/database.sqlite')
# setting up a connection to our sql file in order to read the text data 


# filtering only +ve and -ve reviews i.e
# not taking into consideration those reviews which have Score=3

filtered_data = pd.read_sql_query("""
SELECT * 
FROM Reviews
WHERE Score != 3
""", con)

# Give reviews with score>3 a +ve rating , and review with Score<3  a _ve rating

def partition(x):
    if x < 3:
        return 'Negative'
    return 'Positive'

# Changing reviews with score<3 to be -ve and Vice versa
actualScore = filtered_data['Score']
positiveNegative = actualScore.map(partition)
filtered_data['Score'] = positiveNegative
 

In [2]:
filtered_data.shape
filtered_data.head(10)


,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,Positive,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,Negative,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,Positive,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...
3,4,B000UA0QIQ,A395BORC6FGVXV,Karl,3,3,Negative,1307923200,Cough Medicine,If you are looking for the secret ingredient i...
4,5,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",0,0,Positive,1350777600,Great taffy,Great taffy at a great price. There was a wid...
5,6,B006K2ZZ7K,ADT0SRK1MGOEU,Twoapennything,0,0,Positive,1342051200,Nice Taffy,I got a wild hair for taffy and ordered this f...
6,7,B006K2ZZ7K,A1SP2KVKFXXRU1,David C. Sullivan,0,0,Positive,1340150400,Great! Just as good as the expensive brands!,This saltwater taffy had great flavors and was...
7,8,B006K2ZZ7K,A3JRGQVEQN31IQ,Pamela G. Williams,0,0,Positive,1336003200,"Wonderful, tasty taffy",This taffy is so good. It is very soft and ch...
8,9,B000E7L2R4,A1MZYO9TZK0BBI,R. James,1,1,Positive,1322006400,Yay Barley,Right now I'm mostly just sprouting this so my...
9,10,B00171APVA,A21BT40VZCCYT4,Carol A. Reed,0,0,Positive,1351209600,Healthy Dog Food,This is a very healthy dog food. Good for thei...


In [3]:
filtered_data

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,Positive,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,Negative,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,Positive,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...
3,4,B000UA0QIQ,A395BORC6FGVXV,Karl,3,3,Negative,1307923200,Cough Medicine,If you are looking for the secret ingredient i...
4,5,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",0,0,Positive,1350777600,Great taffy,Great taffy at a great price. There was a wid...
...,...,...,...,...,...,...,...,...,...,...
525809,568450,B001EO7N10,A28KG5XORO54AY,Lettie D. Carter,0,0,Positive,1299628800,Will not do without,Great for sesame chicken..this is a good if no...
525810,568451,B003S1WTCU,A3I8AFVPEE8KI5,R. Sawyer,0,0,Negative,1331251200,disappointed,I'm disappointed with the flavor. The chocolat...
525811,568452,B004I613EE,A121AA1GQV751Z,"pksd ""pk_007""",2,2,Positive,1329782400,Perfect for our maltipoo,"These stars are small, so you can give 10-15 o..."
525812,568453,B004I613EE,A3IBEVCTXKNOH,"Kathy A. Welch ""katwel""",1,1,Positive,1331596800,Favorite Training and reward treat,These are the BEST treats for training and rew...


In [4]:
# we have successfully imported data and did transformations we wanted to do 

# Data Cleaning using Deduplication 




In [5]:
# to see duplicated data 
display = pd.read_sql_query("""
SELECT *
FROM Reviews
WHERE Score != 3 AND UserId="AR5J8UI46CURR"
ORDER By ProductID
""", con)
display

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,78445,B000HDL1RQ,AR5J8UI46CURR,Geetha Krishnan,2,2,5,1199577600,LOACKER QUADRATINI VANILLA WAFERS,DELICIOUS WAFERS. I FIND THAT EUROPEAN WAFERS ...
1,138317,B000HDOPYC,AR5J8UI46CURR,Geetha Krishnan,2,2,5,1199577600,LOACKER QUADRATINI VANILLA WAFERS,DELICIOUS WAFERS. I FIND THAT EUROPEAN WAFERS ...
2,138277,B000HDOPYM,AR5J8UI46CURR,Geetha Krishnan,2,2,5,1199577600,LOACKER QUADRATINI VANILLA WAFERS,DELICIOUS WAFERS. I FIND THAT EUROPEAN WAFERS ...
3,73791,B000HDOPZG,AR5J8UI46CURR,Geetha Krishnan,2,2,5,1199577600,LOACKER QUADRATINI VANILLA WAFERS,DELICIOUS WAFERS. I FIND THAT EUROPEAN WAFERS ...
4,155049,B000PAQ75C,AR5J8UI46CURR,Geetha Krishnan,2,2,5,1199577600,LOACKER QUADRATINI VANILLA WAFERS,DELICIOUS WAFERS. I FIND THAT EUROPEAN WAFERS ...


In [6]:
# here it can be seen that user Geeta Krishnan reviewd for one product but at the same time all other products were reviewed 
# Noting that other products are same as well which means that rating for one product is being duplicated 

In [7]:
# Removing Duplicated  Data 
# Sorting the data in ascending order  according to ProductID
sorted_data=filtered_data.sort_values('ProductId',axis=0,ascending= True, inplace=False, kind='quicksort', na_position='last')

#Dedupliacation of entries
final = sorted_data.drop_duplicates(subset={"UserId","ProfileName","Time","Text"},keep='first',inplace=False)
final.shape

(364173, 10)

In [8]:
# to see how much % of data still remains 
(final['Id'].size*1.0)/(filtered_data['Id'].size*1.0)*100

69.25890143662969

In [9]:
# by observing the data it can also be seen that for some reviews helpfulness num > helpfulness Denominator 
# We have to remove those kinf of entries 

display = pd.read_sql_query("""
SELECT *
FROM Reviews
WHERE Score != 3 AND Id=44737 OR Id=64422
""",con)
display

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,44737,B001EQ55RW,A2V0I904FH7ABY,Ram,3,2,4,1212883200,Pure cocoa taste with crunchy almonds inside,It was almost a 'love at first bite' - the per...
1,64422,B000MIDROQ,A161DK06JJMCYF,"J. E. Stephens ""Jeanne""",3,1,5,1224892800,Bought This for My Son at College,My son loves spaghetti so I didn't hesitate or...


In [10]:
# Code to remove such data 
final=final[final.HelpfulnessNumerator<=final.HelpfulnessDenominator]

In [11]:
print(final.shape)
# How many positive and negative reviews are present in our dataset ?
final['Score'].value_counts()

(364171, 10)


Positive    307061
Negative     57110
Name: Score, dtype: int64

# Data Preprocessing 

In [12]:
# Bag of Words - for conversion of a review(English) into Numerical Vectors.
count_vect = CountVectorizer() # IN -scikit learn 
final_counts = count_vect.fit_transform(final["Text"].values)
# converting each review into a vector of a sparse matrix where the column represent all the words used in this complete data.

In [13]:
type(final_counts)

scipy.sparse.csr.csr_matrix

In [14]:
# Sparse --> it is a sparse matrix i.e most of the rows in these matrix have 0 as their elements 
# and further space complexity of this sparse matrix could be reduced to the number of non_zero cells/ dimensions in the matrix.

In [15]:
final_counts.get_shape()

(364171, 115281)

In [16]:
## Text Pre-processing --> lemmitization, stop words removal and Stemming .



# [3].  Text Preprocessing.


"""Now that we have finished deduplication our data requires some preprocessing before we go on further with analysis and making the prediction model.

Hence in the Preprocessing phase we do the following in the order below:-

1. Begin by removing the html tags
2. Remove any punctuations or limited set of special characters like , or . or # etc.
3. Check if the word is made up of english letters and is not alpha-numeric
4. Check to see if the length of the word is greater than 2 (as it was researched that there is no adjective in 2-letters)
5. Convert the word to lowercase
6. Remove Stopwords
7. Finally Snowball Stemming the word (it was obsereved to be better than Porter Stemming)<br>

After which we collect the words used to describe positive and negative reviews """

In [17]:
import re
#find sentences containing HTML tage 
i = 0
for sent in final["Text"].values:
    if (len(re.findall("<.*?>", sent))):
        print(i)
        print(sent)
        break;
    i += 1
    

6
I set aside at least an hour each day to read to my son (3 y/o). At this point, I consider myself a connoisseur of children's books and this is one of the best. Santa Clause put this under the tree. Since then, we've read it perpetually and he loves it.<br /><br />First, this book taught him the months of the year.<br /><br />Second, it's a pleasure to read. Well suited to 1.5 y/o old to 4+.<br /><br />Very few children's books are worth owning. Most should be borrowed from the library. This book, however, deserves a permanent spot on your shelf. Sendak's best.


In [18]:
# <br>--> break the sentence and go to next line.

In [19]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Lenovo\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [20]:
import re   #re= Regular expressions 
import string 
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer

stop = set(stopwords.words('english'))   # set of stopwords
sno = nltk.stem.SnowballStemmer('english') # for Using Snowball lemmitizer 

def cleanhtml(sentence): #function used to kill any html sentence 
    cleanr = re.compile('<.*?>')   # here <.*?> means to consider all of the sentence in the Corpus 
    cleantext = re.sub('cleanr', ' ',sentence)
    return cleantext
def cleanpunc(sentence):
    cleaned = re.sub(r'[?|!|\'|"|#]',r'',sentence)
    cleaned = re.sub(r'[.|,|)|(|\|/]',r'',cleaned)
    return cleaned
print(stop)
print('******************************************')
print(sno.stem("tasty"))
print(sno.stem("beautiful"))


{'that', 'as', 'how', 'o', 'be', "mightn't", 'shan', "don't", "aren't", 'i', 'then', "you'd", "should've", 'mustn', 'through', 'once', 'shouldn', 'what', 'needn', 'of', "mustn't", 'their', 'above', 'when', 'no', 'here', 'herself', 'can', 'other', 'into', 'and', "that'll", 'we', "wasn't", 'yours', 'couldn', 'do', 'from', 'don', 'with', 'about', 'isn', 'she', 'or', 'd', 'down', 'itself', 'a', 'me', 'll', 'few', 's', 'm', 'ain', 'will', "couldn't", 'up', 'so', 'between', 'is', "you'll", 'did', 'was', 'any', 'ours', 'hers', 're', 'same', 'mightn', 'nor', 'doesn', "shan't", 'had', 'those', 'such', 'where', 'more', 'which', 'in', 'have', 'the', 'being', 'under', 'ourselves', 'having', 'wasn', 'if', 'at', 'again', 'wouldn', 'they', 'only', 'didn', 'very', 'out', "wouldn't", 'been', 'won', "doesn't", 'its', 'until', 'while', 'his', 'theirs', 'some', "needn't", 'all', 'to', 'but', 'too', 'were', 'y', 'my', 'he', 'during', 'haven', 'most', 'each', 'by', 'it', "she's", 'against', 'below', "didn't

In [21]:
# code for implementing step by step checks that was mentioned in preprocessing techniques
i=0
strl=" "
final_string=[ ]
all_positive_words=[ ]
all_negative_words=[ ]
s=' '

for sent in final["Text"].values:
    filtered_sentence=[]
    #print(sent);
    sent = cleanhtml(sent) # remove HTML tags
    for w in sent.split():
        for cleaned_words in cleanpunc(w).split():
            if ((cleaned_words.isalpha()) & (len(cleaned_words)>2)):
                if (cleaned_words.lower() not in stop):
                    s=(sno.stem(cleaned_words.lower())).encode('utf8')
                    filtered_sentence.append(s)
                    if (final['Score'].values)[i] == 'positive':
                        all_positive_words.append(s)
                    if (final['Score'].values)[i] == 'negative':
                        all_negative_words.append(s)
                else:
                    continue
            else:
                continue
    #print(filtered_sentence)
    strl= b" ".join(filtered_sentence) # final string of clean words 
    #print("*************************************************************")
    
    final_string.append(strl)
    i += 1
                        


In [22]:
# Adding a column named Clean Text in the final data 
final["CleanedText"] = final_string

In [23]:
final.head(3)
#storing the final table in sql table for future use
conn = sqlite3.connect('final.sqlite')
c=conn.cursor()
conn.text_factory = str
final.to_sql('Reviews', conn, schema = None, if_exists='replace' )

In [24]:
# Bag of Words - for conversion of a review(English) into Numerical Vectors.
count_vect = CountVectorizer() # IN -scikit learn 
final_counts = count_vect.fit_transform(final["Text"].values)
# converting each review into a vector of a sparse matrix where the column represent all the words used in this complete data.

In [25]:
final_counts.get_shape()

(364171, 115281)

In [26]:
## Bi-Grams and N-Grams 
freq_dist_positive=nltk.FreqDist(all_positive_words)
freq_dist_negative=nltk.FreqDist(all_negative_words)
print("Most Common +ve words: ",freq_dist_positive.most_common(20))
print("Most Common -ve words: ",freq_dist_negative.most_common(20))

Most Common +ve words:  []
Most Common -ve words:  []


In [27]:
# Bi gram , Tri Gram and n gram 
count_vect= CountVectorizer(ngram_range=(1,2) )
final_bigram_counts = count_vect.fit_transform(final['Text'].values)

In [28]:
final_bigram_counts.get_shape()

(364171, 2910192)

# TF-IDF 

In [29]:
tf_idf_vect = TfidfVectorizer(ngram_range=(1,2))
final_tf_idf = tf_idf_vect.fit_transform(final['Text'].values)

In [30]:
final_tf_idf.get_shape()

(364171, 2910192)

In [31]:
features = tf_idf_vect.get_feature_names()
len(features)

2910192

In [32]:
features[100000:100005]

['ales until', 'ales ve', 'ales would', 'ales you', 'alessandra']

In [33]:
# converting a row of a sparsematrix into a numpy array
print(final_tf_idf[100,:].toarray()[0])
# here 100 is the row number which has been converted to a numpy array 

[0. 0. 0. ... 0. 0. 0.]


In [34]:
# using top idf features --> to get top tfidf values in a row and return with corresponding words 
def top_tfidf_feats(row, features, top_n=25):
    topn_ids = np.argsort(row)[::-1][:top_n]
    top_feats = [(features[i],row[i]) for i in topn_ids]
    df = pd.DataFrame(top_feats)
    df.columns = ['feature','tfidf']
    return df

top_tfidf = top_tfidf_feats(final_tf_idf[1,:].toarray()[0],features,25)
    

In [35]:
top_tfidf

,feature,tfidf
0,sendak books,0.173437
1,rosie movie,0.173437
2,paperbacks seem,0.173437
3,cover version,0.173437
4,these sendak,0.173437
5,the paperbacks,0.173437
6,pages open,0.173437
7,really rosie,0.168074
8,incorporates them,0.168074
9,paperbacks,0.168074


# Word 2 Vector  

In [36]:
from gensim.models import Word2Vec
from gensim.models import KeyedVectors
import pickle

unable to import 'smart_open.gcs', disabling that module


In [37]:
# Training our own W2v model using our own text corpus 
import gensim
i=0
list_of_sent=[]
for sent in final['Text'].values:
    filtered_sentence=[]
    sent=cleanhtml(sent)
    for w in sent.split():
        for cleaned_words in cleanpunc(w).split():
            if(cleaned_words.isalpha()):
                filtered_sentence.append(cleaned_words.lower())
            else:
                continue
    list_of_sent.append(filtered_sentence)

In [38]:
print(final['Text'].values(0))
print('******************************************')
print(list_of_sent[0])

TypeError: 'numpy.ndarray' object is not callable

In [ ]:
w2v_model = gensim.models.Word2Vec(list_of_sent,min_count=5,size=50, workers=4)

In [ ]:
words = list(w2v_model.wv.vocab)
print(len(words))

In [ ]:
w2v_model.wv.most_similar('tasty')

In [ ]:
w2v_model.wv.most_similar('woman')

In [ ]:
count_vect_feat = count_vect.get_feature_names()
count_vect_feat.index('like')
print(count_vect_feat[64036])

# Avg W2V, TFIDF-W2V 

In [ ]:
# average Word 2 Vector 
# compute average w2c for each review 
sent_vectors = []; # avg w2v for each sentence is stored in this 
for sent in list_of_sent: # for each review/sentence
    sent_vec = np.zeros(50) # as word vectors are of zero length 
    cnt_words= 0; # num of words with a valid vector in the sentence 
    for word in sent: # for each word in a review
        try:
            vec= w2v_model.wv[word]
            sent_vec += vec 
            cnt_words += 1
        except:
            pass
    sent_vec /= cnt_words
    sent_vectors.append(sent_vec)
print(len(sent_vectors))
print(len(sent_vectors[0]))
        


In [ ]:
# TFIDF-W2V
tfdif_feat = tf_idf_vect.get_feature_names()  # tfidf words/ col- names 
# final tfidf is the sparse matrix with row= sentence, col=word and cell = tfidf value 
tfidf_sent_vectors = []; # tfidf-w2v for each review is stored 
row=0;
for sent in list_of_sent: # for each review
    sent_vec = np.zeros(50) # as word vectors are of zero length 
    weight_sum=0; # num of words with a valid vector in the sentence 
    for word in sent: # for each word in a review
        try:
            vec = w2v_model.wv[word]
            # obtain tfidf of a word in a sentence 
            tfidf= final_tf_idf[row,tfidf_feat.index(word)]
            sent_vec += (vec * tf_idf)
            weight_sum += tf_idf
        except:
            pass
    sent_vec /= weight_sum 
    tfidf_sent_vectors.append(sent_vec)
    row += 1